In [1]:
import plotly.offline as plt
from plotly import tools
import plotly.graph_objs as go
from plotly.graph_objs import Scatter, Layout
from plotly.graph_objs.graph_objs import Bar, Data
from ipywidgets import *
import ipywidgets as widgets
from IPython.display import clear_output
plt.init_notebook_mode(connected=True)


In [5]:
stepList=[[[130,131,132,133,134,135],[1.0,1.1,1.2,1.5,0.3, 1.9]], [[130,131,132,133,134,135], [1.1,1.5,1.0,0.2,1.3,1.8]]]

slider_cuttoff = widgets.FloatSlider(max=5, step=0.001, orientation="horizontal")

textfloat_cuttoff = BoundedFloatText(max=5, min=0, step=0.001)
#display(textfloat_cuttoff,slider_cuttoff)

mylink = widgets.jslink((textfloat_cuttoff, 'value'), (slider_cuttoff, 'value'))

class MultiHist(object):
    """Class for showing stacked histograms using plotly"""
    def __init__(self):
        self.x=[[chr(carac) for carac in range(1,600)], [chr(carac) for carac in range(1,600)], [chr(carac) for carac in range(1,600)]]
        self.y=[[0.1*num for num in range(1,600)], [0.2*num for num in range(1,600)], [0.4*num for num in range(1,600)]]
        self.color=[[10]*len(self.y[0]),[5]*len(self.y[1]), [15]*len(self.y[2])]
        self.layout=None        
        
    def update(self, slider, textbox):
        """This function updates the cutoff value when it is modified by 1 of the 2 types of widget"""
        fig = tools.make_subplots(rows=len(self.y), cols=1, print_grid=False)
        data = []
        x=self.x
        y=self.y
        for stepNumber,step in enumerate(self.y):
            for index,elm in enumerate(step):
                if elm > slider:
                    self.color[stepNumber][index]=[18]
                else:
                    self.color[stepNumber][index]='rgba(204,204,204,1)'
            trace1 = go.Bar(
                    x=self.x[stepNumber],
                    y=self.y[stepNumber],
                    marker=dict(
                        color=self.color[stepNumber]),
                )
            data = [trace1]
            fig.append_trace(trace1, stepNumber+1, 1)
            fig['layout'].update(title='Titration step 1 to '+str(len(self.y)),shapes=[
                                    {
                                        'type': 'line',
                                        'xref': 'paper',
                                        'x0':-1,
                                        'y0': slider, # use absolute value or variable here
                                        'x1':len(step),
                                        'y1': slider, # ditto
                                        'xref':"x",
                                        'yref':"y",
                                        'line': {
                                            'color': 'rgb(50, 171, 96)',
                                            'width': 2,

                                        },
                                    },
                                ]*len(self.y)
                )
            
        for index,step in enumerate(self.y):
            fig['layout']['shapes'][index]['xref']="x"+str(index+1)
            fig['layout']['shapes'][index]['yref']="y"+str(index+1)
        plt.iplot(fig)

h=MultiHist()      
widgets.interactive(h.update, slider=slider_cuttoff, textbox=textfloat_cuttoff)

A Jupyter Widget

In [48]:
stepList=[[[130,131,132,133,134,135],[1.0,1.1,1.2,1.5,0.3]], [[130,131,132,133,134,135], [1.1,1.5,1.0,0.2,1.3,1.8]]]

fig = tools.make_subplots(rows=round(len(stepList)/2), cols=2)
for index,elm in enumerate(stepList):
    trace= go.Scatter(x=[aminoacid for aminoacid in elm[0]], y=[intensity for intensity in elm[1]], marker=Marker(
            size=16,
            cmax=39,
            cmin=0,
            color=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,]))
    fig.append_trace(trace, 1, index+1 )

fig['layout'].update(height=600, width=600, title='Multiple Subplots' +
                                                  ' with Titles')

plt.iplot(fig, filename='make-subplots-multiple-with-titles')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



NameError: name 'Marker' is not defined